# **Worksapce set up**

In [33]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [34]:
# Import libraries
%pylab inline
!pip install -q tensorflow-model-optimization

import tensorflow as tf
import tensorflow_model_optimization as tfmot
import numpy as np
import tempfile
import zipfile
import os
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

Populating the interactive namespace from numpy and matplotlib


In [35]:
# Display python and library versions
!python --versions
print('Numpy ' + np.__version__)
print('TensorFlow ' + tf.__version__)
print('Keras ' + tf.keras.__version__)

unknown option --versions
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
Numpy 1.21.6
TensorFlow 2.8.2
Keras 2.8.0


# **Dataset Management**

In [36]:
# Loads the data and splits it into 60% training and 40% testing sets
df = loadtxt("drive/MyDrive/GE_practicum/diabetes.csv", delimiter=",", skiprows=1)

x_train = df[0:int(0.6*len(df)),0:8]
y_train = df[0:int(0.6*len(df)),8]

x_test = df[int(0.6*len(df)):,0:8]
y_test = df[int(0.6*len(df)):,8]

# **Clustering**

In [37]:
# Loads the base model for ANN tests and gets baseline accuracy for result comparison
base_model = tf.keras.models.load_model('drive/MyDrive/GE_practicum/ANN_base')
base_model.fit(x_train,y_train,epochs = 50,validation_data = (x_test,y_test), batch_size=10)
base_model.summary()

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(base_model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Epoch 1/50
46/46 [==============================] - 1s 8ms/step - loss: 0.5617 - accuracy: 0.7152 - val_loss: 0.6265 - val_accuracy: 0.7175
Epoch 2/50
46/46 [==============================] - 0s 6ms/step - loss: 0.4966 - accuracy: 0.7522 - val_loss: 0.7638 - val_accuracy: 0.5552
Epoch 3/50
46/46 [==============================] - 0s 5ms/step - loss: 0.5022 - accuracy: 0.7543 - val_loss: 0.6645 - val_accuracy: 0.5942
Epoch 4/50
46/46 [==============================] - 0s 4ms/step - loss: 0.5173 - accuracy: 0.7543 - val_loss: 0.5966 - val_accuracy: 0.7532
Epoch 5/50
46/46 [==============================] - 0s 5ms/step - loss: 0.5248 - accuracy: 0.7457 - val_loss: 0.5855 - val_accuracy: 0.7273
Epoch 6/50
46/46 [==============================] - 0s 4ms/step - loss: 0.5096 - accuracy: 0.7609 - val_loss: 0.6195 - val_accuracy: 0.7110
Epoch 7/50
46/46 [==============================] - 0s 9ms/step - loss: 0.5074 - accuracy: 0.7522 - val_loss: 0.5859 - val_accuracy: 0.7338
Epoch 8/50
46/46 [==

In [38]:
cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

clustered_model = cluster_weights(base_model, **clustering_params)

opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
clustered_model.compile(loss="binary_crossentropy", optimizer=opt, metrics=['accuracy'])
clustered_model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cluster_dense_8 (ClusterWei  (None, 128)              2192      
 ghts)                                                           
                                                                 
 cluster_dense_9 (ClusterWei  (None, 64)               16464     
 ghts)                                                           
                                                                 
 cluster_dense_10 (ClusterWe  (None, 32)               4144      
 ights)                                                          
                                                                 
 cluster_dense_11 (ClusterWe  (None, 1)                81        
 ights)                                                          
                                                                 
Total params: 22,881
Trainable params: 11,585
Non-trai

In [39]:
clustered_model.fit(x_train,y_train,epochs = 10,validation_data = (x_test,y_test), batch_size=10)
clustered_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(clustered_model, clustered_keras_file, include_optimizer=False)
print('Saved clustered Keras model to:', clustered_keras_file)

Epoch 1/10
46/46 [==============================] - 2s 17ms/step - loss: 0.5933 - accuracy: 0.6978 - val_loss: 0.6452 - val_accuracy: 0.6851
Epoch 2/10
46/46 [==============================] - 1s 16ms/step - loss: 0.5781 - accuracy: 0.7087 - val_loss: 0.6406 - val_accuracy: 0.6851
Epoch 3/10
46/46 [==============================] - 1s 12ms/step - loss: 0.5684 - accuracy: 0.7130 - val_loss: 0.6377 - val_accuracy: 0.6883
Epoch 4/10
46/46 [==============================] - 1s 13ms/step - loss: 0.5598 - accuracy: 0.7152 - val_loss: 0.6384 - val_accuracy: 0.6916
Epoch 5/10
46/46 [==============================] - 1s 14ms/step - loss: 0.5494 - accuracy: 0.7261 - val_loss: 0.6268 - val_accuracy: 0.7013
Epoch 6/10
46/46 [==============================] - 1s 17ms/step - loss: 0.5396 - accuracy: 0.7239 - val_loss: 0.6404 - val_accuracy: 0.6883
Epoch 7/10
46/46 [==============================] - 1s 13ms/step - loss: 0.5360 - accuracy: 0.7239 - val_loss: 0.6252 - val_accuracy: 0.7208
Epoch 8/10
46

Saved clustered Keras model to: /tmp/tmp1482hgmd.h5


# **Quantization aware training**

In [40]:
quantize_model = tfmot.quantization.keras.quantize_model
model = quantize_model(clustered_model)

opt = tf.keras.optimizers.Adam(lr = 1e-3,decay = 1e-5)
model.compile(optimizer = opt,loss = "binary_crossentropy" , metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_2 (QuantizeL  (None, 8)                3         
 ayer)                                                           
                                                                 
 quant_dense_8 (QuantizeWrap  (None, 128)              1157      
 perV2)                                                          
                                                                 
 quant_dense_9 (QuantizeWrap  (None, 64)               8261      
 perV2)                                                          
                                                                 
 quant_dense_10 (QuantizeWra  (None, 32)               2085      
 pperV2)                                                         
                                                                 
 quant_dense_11 (QuantizeWra  (None, 1)               

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [41]:
# Performs quantization aware training
model.fit(x_train,y_train,epochs = 50,validation_data = (x_test,y_test), batch_size=10)

Epoch 1/50
46/46 [==============================] - 3s 21ms/step - loss: 0.5562 - accuracy: 0.7109 - val_loss: 0.5746 - val_accuracy: 0.7240
Epoch 2/50
46/46 [==============================] - 1s 11ms/step - loss: 0.5019 - accuracy: 0.7457 - val_loss: 0.5859 - val_accuracy: 0.7305
Epoch 3/50
46/46 [==============================] - 0s 10ms/step - loss: 0.4883 - accuracy: 0.7609 - val_loss: 0.6264 - val_accuracy: 0.6494
Epoch 4/50
46/46 [==============================] - 0s 10ms/step - loss: 0.4738 - accuracy: 0.7522 - val_loss: 0.6499 - val_accuracy: 0.6851
Epoch 5/50
46/46 [==============================] - 0s 10ms/step - loss: 0.4732 - accuracy: 0.7717 - val_loss: 0.6089 - val_accuracy: 0.7305
Epoch 6/50
46/46 [==============================] - 1s 12ms/step - loss: 0.4484 - accuracy: 0.7761 - val_loss: 0.5722 - val_accuracy: 0.7403
Epoch 7/50
46/46 [==============================] - 1s 16ms/step - loss: 0.4415 - accuracy: 0.7739 - val_loss: 0.6551 - val_accuracy: 0.7013
Epoch 8/50
46

# **Quantization and conversion to tflite**

In [42]:
# Conversion to tflite

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS
]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

open('ANN_clustered.tflite', 'wb').write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpw1aigb6u/assets


INFO:tensorflow:Assets written to: /tmp/tmpw1aigb6u/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


15736

In [43]:
# Check tflite model characteristics
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)

=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the QUANTIZE op takes
tensor #0 as input and produces tensor #5 as output.

Subgraph#0 main(T#0) -> [T#15]
  Op#0 QUANTIZE(T#0) -> [T#5]
  Op#1 FULLY_CONNECTED(T#5, T#6, T#3) -> [T#7]
  Op#2 FULLY_CONNECTED(T#7, T#8, T#4) -> [T#9]
  Op#3 FULLY_CONNECTED(T#9, T#10, T#1) -> [T#11]
  Op#4 FULLY_CONNECTED(T#11, T#12, T#2) -> [T#13]
  Op#5 LOGISTIC(T#13) -> [T#14]
  Op#6 DEQUANTIZE(T#14) -> [T#15]

Tensors of Subgraph#0
  T#0(serving_default_dense_8_input:0) shape_signature:[-1, 8], type:FLOAT32
  T#1(dense_10/bias) shape:[32], type:INT32 RO 128 bytes
  T#2(dense_11/bias) shape:[1], type:INT32 RO 4 bytes
  T#3(dense_8/bias) shape:[128], type:INT32 RO 512 bytes
  T#4(dense_9/bias) shape:[64], type:INT32 RO 256 bytes
  T#5(sequential_2/quantize_layer_2/AllValuesQuantize/FakeQuantWithMinMaxVars;quantize_layer_2/quantize_layer_2_mi

In [44]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
input_details = interpreter.get_input_details()

interpreter.allocate_tensors()
output_details = interpreter.get_output_details()

#Predictions from TFLite model
tfl_pred = []
tfl_pred_class = []
for i in range(len(x_test)):
    interpreter.set_tensor(input_details[0]["index"], x_test.astype('float32')[i:i+1,:])
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])
    tfl_pred.append(result)
    tfl_pred_class.append(int(result[0]*2))

right_pred = [y_test[i] == tfl_pred_class[i] for i in range(len(y_test))]
acc = sum(right_pred)/len(right_pred)
print(acc)

0.6818181818181818


# **Conversion to C array**

In [45]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str

In [46]:
# Write TFLite model to a C source (or header) file
with open("ANN_clustered" + '.h', 'w') as file:
  file.write(hex_to_c_array(tflite_model, "ANN_clustered"))

# **Size Comparison**

In [47]:
def get_gzipped_model_size(file):
  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [48]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))
print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size('ANN_clustered.tflite')))

Size of gzipped baseline Keras model: 45027.00 bytes
Size of gzipped clustered Keras model: 9860.00 bytes
Size of gzipped clustered TFlite model: 11003.00 bytes
